In [51]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import xgboost
import lightgbm

In [52]:
X_artificial=pd.read_csv("../data/artificial/artificial_train.data",sep=" ",header=None)
y_artificial=pd.read_csv("../data/artificial/artificial_train.labels",sep=" ",header=None)
X_digits=pd.read_csv("../data/digits/digits_train.data",sep=" ",header=None)
y_digits=pd.read_csv("../data/digits/digits_train.labels",sep=" ",header=None)

X_artificial.drop(columns=500,inplace=True)
X_digits.drop(columns=5000,inplace=True)

VT_art = VarianceThreshold(threshold = 0)
VT_art.fit(X_artificial)
X_artificial=X_artificial.iloc[:,VT_art.get_support(indices=True).tolist()]

VT_dig = VarianceThreshold(threshold = 0)
VT_dig.fit(X_digits)
X_digits=X_digits.iloc[:,VT_dig.get_support(indices=True).tolist()]
print(X_digits.shape,X_artificial.shape)



(6000, 4955) (2000, 500)


In [53]:
X_digits_train,X_digits_test,y_digits_train,y_digits_test=train_test_split(X_digits,y_digits,stratify=y_digits,test_size=0.25)

y_digits_train=y_digits_train.to_numpy().ravel()

y_digits_test=y_digits_test.to_numpy().ravel()

### no feature rescaling, no feature selection

In [54]:
logistic_model=LogisticRegression(penalty="l2",tol=1e-5,max_iter=10000)
logistic_model.fit(X_digits_train,y_digits_train)
(logistic_model.predict(X_digits_test)==y_digits_test).mean()

0.966

In [55]:
random_forest_model=RandomForestClassifier(n_jobs=-1) #do sprawdzenia parametry
random_forest_model.fit(X_digits_train,y_digits_train)
(random_forest_model.predict(X_digits_test)==y_digits_test).mean()

0.962

In [56]:
xgb_model = xgboost.XGBClassifier() #do sprawdzenia parametry
xgb_model.fit(X_digits_train,(y_digits_train+1)/2)
(xgb_model.predict(X_digits_test)==(y_digits_test+1)/2).mean()

0.9773333333333334

In [57]:
adaboost_model=AdaBoostClassifier(n_estimators=100)
adaboost_model.fit(X_digits_train,y_digits_train)
(adaboost_model.predict(X_digits_test)==y_digits_test).mean(),sum(adaboost_model.feature_importances_>0)

(0.9673333333333334, 95)

In [58]:
lgm_model=lightgbm.LGBMClassifier()
lgm_model.fit(X_digits_train,y_digits_train)
(lgm_model.predict(X_digits_test)==y_digits_test).mean(),sum(lgm_model.feature_importances_>0)

(0.9753333333333334, 971)

### feature rescaling, no feature selection

In [59]:
ss=StandardScaler()
ss.fit(X_digits_train)
X_digits_train_sc=ss.transform(X_digits_train)
X_digits_test_sc=ss.transform(X_digits_test)

In [60]:
logistic_model=LogisticRegression(penalty="l2",tol=1e-5,max_iter=1000)
logistic_model.fit(X_digits_train_sc,y_digits_train)
(logistic_model.predict(X_digits_test_sc)==y_digits_test).mean()

0.9713333333333334

In [61]:
logistic_model=LogisticRegression(penalty="l1",tol=1e-5,max_iter=1000,solver='liblinear')
logistic_model.fit(X_digits_train_sc,y_digits_train)
(logistic_model.predict(X_digits_test_sc)==y_digits_test).mean(),(logistic_model.coef_>0).sum()

(0.976, 263)

In [62]:
random_forest_model=RandomForestClassifier(n_jobs=-1) #do sprawdzenia parametry
random_forest_model.fit(X_digits_train_sc,y_digits_train)
(random_forest_model.predict(X_digits_test_sc)==y_digits_test).mean()

0.964

In [63]:
xgb_model = xgboost.XGBClassifier() #do sprawdzenia parametry
xgb_model.fit(X_digits_train_sc,(y_digits_train+1)/2)
(xgb_model.predict(X_digits_test_sc)==(y_digits_test+1)/2).mean(),sum(xgb_model.feature_importances_>0)

(0.9773333333333334, 680)

In [64]:
adaboost_model=AdaBoostClassifier(n_estimators=100)
adaboost_model.fit(X_digits_train_sc,y_digits_train)
(adaboost_model.predict(X_digits_test_sc)==y_digits_test).mean(),sum(adaboost_model.feature_importances_>0)

(0.9673333333333334, 95)

In [65]:
lgm_model=lightgbm.LGBMClassifier()
lgm_model.fit(X_digits_train_sc,y_digits_train)
(lgm_model.predict(X_digits_test_sc)==y_digits_test).mean(),sum(lgm_model.feature_importances_>0)

(0.9746666666666667, 974)

In [71]:
%load_ext autoreload
%autoreload 2
from eval_models import eval_models

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
